In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Metabolic_Rate"
cohort = "GSE151683"

# Input paths
in_trait_dir = "../../input/GEO/Metabolic_Rate"
in_cohort_dir = "../../input/GEO/Metabolic_Rate/GSE151683"

# Output paths
out_data_file = "../../output/preprocess/Metabolic_Rate/GSE151683.csv"
out_gene_data_file = "../../output/preprocess/Metabolic_Rate/gene_data/GSE151683.csv"
out_clinical_data_file = "../../output/preprocess/Metabolic_Rate/clinical_data/GSE151683.csv"
json_path = "../../output/preprocess/Metabolic_Rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE151683_family.soft.gz', 'GSE151683_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Metabolic_Rate/GSE151683/GSE151683_family.soft.gz
Matrix file: ../../input/GEO/Metabolic_Rate/GSE151683/GSE151683_series_matrix.txt.gz
Background Information:
!Series_title	"DNA microarray analysis of blood before and after ingesting carotenoid-rich vegitable beverage for 8 weeks, a randomized and double-blinded controlled clinical trial"
!Series_summary	"Metabolic syndrome, whose main diagnostic component is obesity, is a risk factor for lifestyle-related diseases, type 2 diabetes, and cardiovascular disease. Diet is known to affect the prevalence of metabolic syndrome. However, the effect of diet on metabolic syndrome in Japanese subjects has not been thoroughly explored. In the present study, we investigated the effect of carotenoid-rich vegetables, particularly lycopene- and lutein-rich vegetables, on the metabolic syndrome in obese Japanese men. We conducted an 8-week 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Dict, Any, Optional, Callable
import gzip

# 1. Gene Expression Data Availability
# Based on the background information, this is a DNA microarray analysis study
is_gene_available = True  # DNA microarray data should contain gene expression

# 2. Variable Availability and Data Type Conversion
# 2.1 & 2.2 For Trait (Metabolic Rate)
# From the background, this study focuses on metabolic syndrome and obesity
# The most relevant trait here is the treatment group which affects metabolic rate
trait_row = 3  # treatment group

# The trait is categorical (4 types of treatments) but we need to convert to binary/continuous
def convert_trait(val: str) -> Optional[int]:
    if val is None or not isinstance(val, str):
        return None
    
    value = val.split(': ')[-1].strip().lower() if ': ' in val else val.strip().lower()
    
    # Converting to binary based on lycopene levels (high=1, low=0)
    if 'high lycopene' in value:
        return 1
    elif 'low lycopene' in value:
        return 0
    return None

# 2.1 & 2.2 For Age
age_row = 2  # age is stored at index 2

def convert_age(val: str) -> Optional[float]:
    if val is None or not isinstance(val, str):
        return None
    
    value = val.split(': ')[-1].strip() if ': ' in val else val.strip()
    try:
        return float(value)
    except ValueError:
        return None

# 2.1 & 2.2 For Gender
# The background info and sample characteristics show all participants are male
gender_row = 1  # gender is stored at index 1, though it's constant (all male)

def convert_gender(val: str) -> Optional[int]:
    if val is None or not isinstance(val, str):
        return None
    
    value = val.split(': ')[-1].strip().lower() if ': ' in val else val.strip().lower()
    if value == 'male':
        return 1
    elif value == 'female':
        return 0
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Construct clinical data from the sample characteristics dictionary provided in previous step
    # Sample Characteristics Dictionary format was shown in the output from previous step
    sample_chars = {
        0: ['tissue: whole blood'], 
        1: ['gender: Male'], 
        2: ['age (years): 42', 'age (years): 63', 'age (years): 40', 'age (years): 41', 
            'age (years): 43', 'age (years): 45', 'age (years): 56', 'age (years): 46', 
            'age (years): 62', 'age (years): 44', 'age (years): 54', 'age (years): 48', 
            'age (years): 60', 'age (years): 49'], 
        3: ['treatment group: low lycopene, low lutein', 'treatment group: low lycopene, high lutein', 
            'treatment group: high lycopene, low lutein', 'treatment group: high lycopene, high lutein'], 
        4: ['time (weeks): 0', 'time (weeks): 8'], 
        5: ['subject id: LLyLLu_04', 'subject id: LLyLLu_12', 'subject id: LLyLLu_21', 
            'subject id: LLyLLu_23', 'subject id: LLyLLu_34', 'subject id: LLyHLu_05', 
            'subject id: LLyHLu_11', 'subject id: LLyHLu_19', 'subject id: LLyHLu_24', 
            'subject id: LLyHLu_27', 'subject id: LLyHLu_31', 'subject id: HLyLLu_03', 
            'subject id: HLyLLu_15', 'subject id: HLyLLu_30', 'subject id: HLyLLu_33', 
            'subject id: HLyLLu_37', 'subject id: HLyHLu_01', 'subject id: HLyHLu_07', 
            'subject id: HLyHLu_08', 'subject id: HLyHLu_18', 'subject id: HLyHLu_22', 
            'subject id: HLyHLu_35', 'subject id: HLyHLu_36']
    }
    
    # Load the GSE matrix to get the sample IDs in the correct order
    sample_ids = []
    with gzip.open(os.path.join(in_cohort_dir, "GSE151683_series_matrix.txt.gz"), 'rt') as f:
        for line in f:
            if line.startswith("!Sample_geo_accession"):
                sample_ids = [s.strip() for s in line.strip().split('\t')[1:]]
                break
    
    # Create a DataFrame to store clinical data
    clinical_data = pd.DataFrame(index=sample_ids)
    
    # Read the sample characteristic data
    with gzip.open(os.path.join(in_cohort_dir, "GSE151683_series_matrix.txt.gz"), 'rt') as f:
        reading_chars = False
        char_row = 0
        
        for line in f:
            if line.startswith("!Sample_characteristics_ch1"):
                values = line.strip().split('\t')[1:]  # Skip the header
                
                # Add the data to the DataFrame
                for i, sample_id in enumerate(sample_ids):
                    if i < len(values):
                        clinical_data.loc[sample_id, char_row] = values[i]
                
                char_row += 1
            elif reading_chars and not line.startswith("!Sample_characteristics_ch1"):
                break
            
            if line.startswith("!Sample_characteristics_ch1"):
                reading_chars = True
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Save to CSV
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{0: [nan, nan, nan], 1: [nan, nan, nan], 2: [nan, nan, nan], 3: [0.0, nan, nan], 4: [nan, nan, nan], 5: [nan, nan, nan]}


Clinical data saved to ../../output/preprocess/Metabolic_Rate/clinical_data/GSE151683.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 54675
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers in the gene expression data appear to be Affymetrix microarray probe IDs
# (e.g., '1007_s_at', '1053_at', etc.) rather than standard human gene symbols.
# These probe IDs need to be mapped to official gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe gene identifiers and dictionary preview
# From the gene annotation preview, we can see:
# 'ID' column contains probe IDs like '1007_s_at' which match the gene expression data indices
# 'Gene Symbol' column contains the human gene symbols we need (e.g., 'DDR1 /// MIR4640')

# 2. Create gene mapping dataframe
prob_col = 'ID'  # Column containing probe IDs
gene_col = 'Gene Symbol'  # Column containing gene symbols

# Extract mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene expression
# Get the gene expression data from the matrix file
gene_expression_data = get_genetic_data(matrix_file)

# Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_expression_data, gene_mapping)

print("\nGene data (after mapping) preview:")
print(f"Shape: {gene_data.shape}")
print(f"Number of genes: {len(gene_data.index)}")
print(f"First 5 gene symbols: {list(gene_data.index[:5])}")

# Save gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}



Gene data (after mapping) preview:
Shape: (21278, 46)
Number of genes: 21278
First 5 gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1']


Gene expression data saved to ../../output/preprocess/Metabolic_Rate/gene_data/GSE151683.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Re-examine the clinical data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print out the sample characteristics to verify available rows
characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample characteristics dictionary:")
print(characteristics_dict)

# Define conversion functions for the clinical features based on the actual data
def convert_trait(value):
    """Convert treatment group to binary based on lycopene level (low=0, high=1)."""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if 'high lycopene' in value:
        return 1.0  # High lycopene
    elif 'low lycopene' in value:
        return 0.0  # Low lycopene
    else:
        return None

def convert_age(value):
    """Convert age data to continuous values."""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (female=0, male=1)."""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if 'male' in value:
        return 1.0
    elif 'female' in value:
        return 0.0
    else:
        return None

# Create the clinical dataframe using the correct row indices based on sample characteristics
try:
    # Row 3 contains treatment group (high/low lycopene) - this is our trait of interest
    # Row 2 contains age information
    # Row 1 contains gender information
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait="Metabolic_Rate",  # Using this as the trait name as per variable definition
        trait_row=3,  # Treatment group as the trait (based on lycopene levels)
        convert_trait=convert_trait,
        gender_row=1,  # Gender information is available in row 1
        convert_gender=convert_gender,
        age_row=2,     # Age information is available in row 2
        convert_age=convert_age
    )
    
    print("Clinical data preview:")
    print(preview_df(clinical_df.T))  # Transpose for better viewing
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Saved clinical data to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_normalized)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 4. Handle missing values in the linked data
    linked_data_cleaned = handle_missing_values(linked_data, 'Metabolic_Rate')
    print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")
    
    # 5. Check if the trait and demographic features are biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Metabolic_Rate')
    
    # 6. Validate the dataset and save cohort information
    note = "Dataset contains gene expression data from a study examining the effects of carotenoid-rich vegetables on metabolic syndrome in obese Japanese men. The trait variable is treatment group based on lycopene levels (0=Low lycopene, 1=High lycopene)."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")
        
except Exception as e:
    print(f"Error in processing clinical data: {e}")
    # If we failed to extract clinical data, update the cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=pd.DataFrame(),
        note="Failed to extract clinical data. Gene expression data is available but missing trait information."
    )
    print("Dataset validation failed due to missing clinical data. Only gene data saved.")

Shape of gene data after normalization: (19845, 46)


Saved normalized gene data to ../../output/preprocess/Metabolic_Rate/gene_data/GSE151683.csv
Sample characteristics dictionary:
{0: ['tissue: whole blood'], 1: ['gender: Male'], 2: ['age (years): 42', 'age (years): 63', 'age (years): 40', 'age (years): 41', 'age (years): 43', 'age (years): 45', 'age (years): 56', 'age (years): 46', 'age (years): 62', 'age (years): 44', 'age (years): 54', 'age (years): 48', 'age (years): 60', 'age (years): 49'], 3: ['treatment group: low lycopene, low lutein', 'treatment group: low lycopene, high lutein', 'treatment group: high lycopene, low lutein', 'treatment group: high lycopene, high lutein'], 4: ['time (weeks): 0', 'time (weeks): 8'], 5: ['subject id: LLyLLu_04', 'subject id: LLyLLu_12', 'subject id: LLyLLu_21', 'subject id: LLyLLu_23', 'subject id: LLyLLu_34', 'subject id: LLyHLu_05', 'subject id: LLyHLu_11', 'subject id: LLyHLu_19', 'subject id: LLyHLu_24', 'subject id: LLyHLu_27', 'subject id: LLyHLu_31', 'subject id: HLyLLu_03', 'subject id: HL

Shape of linked data after handling missing values: (46, 19848)
For the feature 'Metabolic_Rate', the least common label is '0.0' with 22 occurrences. This represents 47.83% of the dataset.
The distribution of the feature 'Metabolic_Rate' in this dataset is fine.

Quartiles for 'Age':
  25%: 42.0
  50% (Median): 44.0
  75%: 48.75
Min: 40.0
Max: 63.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 46 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Saved processed linked data to ../../output/preprocess/Metabolic_Rate/GSE151683.csv
